<a href="https://colab.research.google.com/github/cicl-iscl/LeWiDi_SemEval2023/blob/main/Notebooks/CompLingNoTrainerAllDataWeightedAndBalancedLossAndEvaluate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Notes

ConvAbuse is just the dictionary as a string lol

In [1]:
!pip install transformers
!pip install datasets
!pip install evaluate
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 14.8 MB/s 
     |████████████████████████████████| 7.6 MB 55.7 MB/s 
     |████████████████████████████████| 182 kB 43.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 452 kB 9.4 MB/s 
     |████████████████████████████████| 213 kB 32.0 MB/s 
     |████████████████████████████████| 132 kB 37.2 MB/s 
     |████████████████████████████████| 127 kB 44.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 81 kB 7.2 MB/s 


In [2]:
import pandas as pd

filepaths = ["/content/ArMIS_train.json", "/content/ConvAbuse_train.json", "/content/HS-Brexit_train.json", "/content/MD-Agreement_train.json", "/content/rom_train.json"]
dfs = [pd.read_json(filepath) for filepath in filepaths]
dfs = [df.transpose() for df in dfs]
dfs = [df.astype({"hard_label": int}, errors='raise') for df in dfs]#other way aroud (do concat first) would make more sense

df_all = pd.concat(dfs)
#df_all["soft_label"] =  df_all["soft_label"].apply(lambda x: [x["0"], x["1"]])
#should probably do that, but then evaluate script has to be adapted

# Dataset

In [3]:
from datasets import Dataset
dataset = Dataset.from_pandas(df_all[["text", "hard_label", "soft_label"]])

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased", do_lower_case=True)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [5]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=240)#This is the longest that roberta tokenizer would go, dunno

tokenized_datasets = dataset.map(tokenize_function, batched=False)

  0%|          | 0/38685 [00:00<?, ?ex/s]

In [6]:
tokenized_datasets = tokenized_datasets.train_test_split(test_size=0.1)

In [7]:
tokenized_datasets.set_format("torch")#not sure if neccessary

In [8]:
from torch.utils.data import DataLoader

batch_size = 4
train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True, batch_size=batch_size)
eval_dataloader = DataLoader(tokenized_datasets["test"], batch_size=batch_size)

In [9]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)

Downloading:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [10]:
from torch.optim import AdamW
optimizer = AdamW(model.parameters())

In [11]:
from transformers import get_scheduler
num_epochs = 2
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [12]:
import torch
from tqdm import tqdm_notebook
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
m = model.to(device)

In [13]:
import evaluate

#TODO something is off here, untrained model gets 0.6 im ce eval

def evaluate_f1_score_huggingface(model, eval_dataloader):
  total = 0
  f1 = 0
  f1_metric = evaluate.load("f1")
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):
    inputs, attention_mask, hard_labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["hard_label"].to(device)
    with torch.no_grad():
      outputs = model(input_ids=inputs, attention_mask = attention_mask)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    N = predictions.shape[0]
    total+=1#the metric already gives the average, so no + batch_size
    f1 += f1_metric.compute(predictions=predictions, references=hard_labels, average = 'micro')["f1"]#micro must be used, otherwise smth wrong is returned, TODO look into that (00 00 is undefined)

  return f1/total


#slightly different to the pytorch implementations, not sure why TODO
def evaluate_ce_score(model, eval_dataloader, epsilon = 1e-12):
  total = 0
  ce = 0
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):
    inputs, attention_mask, hard_labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["hard_label"].to(device)
    soft_labels = torch.cat([batch["soft_label"]["0"].unsqueeze(0), batch["soft_label"]["1"].unsqueeze(0)], dim=0).T.to(device)
    with torch.no_grad():
      outputs = model(input_ids=inputs, attention_mask = attention_mask)
    logits = outputs.logits
    probabilities = torch.softmax(logits, axis=-1)
    predictions = torch.clip(probabilities, epsilon, 1. - epsilon)                                      
    N = predictions.shape[0]
    total+=N#maybe should be 1 here as well
    ce += -torch.sum(soft_labels*torch.log(predictions+1e-9))
  return ce/total


def evaluate_bce_score_pytorch(model, eval_dataloader):
  criterion = torch.nn.BCELoss(reduction="sum")
  total = 0
  ce = 0
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):
    inputs, attention_mask, hard_labels, soft_labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["hard_label"].to(device), batch["soft_label"]["1"].to(device)
    with torch.no_grad():
      outputs = model(input_ids=inputs, attention_mask = attention_mask)
    logits = outputs.logits
    probabilities = torch.softmax(logits, axis=-1)
    N = probabilities.shape[0]
    total +=N
    ce += criterion(probabilities[:,-1], soft_labels)
  return ce/total


def evaluate_ce_score_pytorch(model, eval_dataloader):
  criterion = torch.nn.CrossEntropyLoss(reduction="sum")
  total = 0
  ce = 0
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):
    inputs, attention_mask, hard_labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["hard_label"].to(device)
    soft_labels = torch.cat([batch["soft_label"]["0"].unsqueeze(0), batch["soft_label"]["1"].unsqueeze(0)], dim=0).T.to(device)
    with torch.no_grad():
      outputs = model(input_ids=inputs, attention_mask = attention_mask)
    logits = outputs.logits
    probabilities = torch.softmax(logits, axis=-1)                          
    N = probabilities.shape[0]
    total+=N
    ce += criterion(probabilities, soft_labels)
  return ce/total



In [14]:
#TODO do this for each dataset independently

#weight = df_all.mean(axis=0)["hard_label"]
weight = df_all["hard_label"].mean()
weight = torch.tensor([weight, 1 - weight], device = device, dtype=torch.float32)#I think this is the right way around

In [15]:

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, batch in enumerate(tqdm_notebook(train_dataloader, 0)):
      #there is also index_level_0 in the dataset, not sure if needed
      inputs, attention_mask, hard_labels, soft_labels = batch["input_ids"].to(device), batch["attention_mask"].to(device), batch["hard_label"].to(device), batch["soft_label"]
      #soft_labels = torch.cat([soft_labels["0"].unsqueeze(-1), soft_labels["1"].unsqueeze(-1)], dim=-1).to(device)# that can probably be done more intuitively
      outputs = model(input_ids=inputs, attention_mask = attention_mask)#NllLoss
      optimizer.zero_grad()
      loss = 0
      for j in range(outputs.logits.shape[0]):
        criterion = criterion = torch.nn.CrossEntropyLoss(weight = (torch.tensor([soft_labels["0"][j], soft_labels["1"][j]], device=device) + weight) / 2)#can probably be done more efficient
        loss += criterion(outputs.logits, hard_labels)
      loss = loss/batch_size
      loss.backward()
      optimizer.step()
      lr_scheduler.step()
      

      running_loss += loss.item()
      if i % 10 == 9:    # print every 2000 mini-batches
        print('[%d, %5d] loss: %.3f' %
              (epoch + 1, i + 1, running_loss / 10))
        running_loss = 0.0

<ipython-input-15-12eeacebd040>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, batch in enumerate(tqdm_notebook(train_dataloader, 0)):


  0%|          | 0/8704 [00:00<?, ?it/s]

[1,    10] loss: 0.909
[1,    20] loss: 1.139
[1,    30] loss: 0.480
[1,    40] loss: 0.659
[1,    50] loss: 0.445
[1,    60] loss: 0.563
[1,    70] loss: 0.468
[1,    80] loss: 0.373
[1,    90] loss: 0.552
[1,   100] loss: 0.597
[1,   110] loss: 0.695
[1,   120] loss: 0.441
[1,   130] loss: 0.606
[1,   140] loss: 0.366
[1,   150] loss: 0.310
[1,   160] loss: 0.417
[1,   170] loss: 0.400
[1,   180] loss: 0.584
[1,   190] loss: 0.389
[1,   200] loss: 0.454
[1,   210] loss: 0.368
[1,   220] loss: 0.510
[1,   230] loss: 0.513
[1,   240] loss: 0.540
[1,   250] loss: 0.369
[1,   260] loss: 0.473
[1,   270] loss: 0.584
[1,   280] loss: 0.562
[1,   290] loss: 0.700
[1,   300] loss: 0.358
[1,   310] loss: 0.197
[1,   320] loss: 0.769
[1,   330] loss: 0.395
[1,   340] loss: 0.698
[1,   350] loss: 0.562
[1,   360] loss: 0.537
[1,   370] loss: 0.701
[1,   380] loss: 0.313
[1,   390] loss: 0.478
[1,   400] loss: 0.292
[1,   410] loss: 0.498
[1,   420] loss: 0.550
[1,   430] loss: 0.500
[1,   440] 

  0%|          | 0/8704 [00:00<?, ?it/s]

[2,    10] loss: 0.309
[2,    20] loss: 0.343
[2,    30] loss: 0.547
[2,    40] loss: 0.386
[2,    50] loss: 0.210
[2,    60] loss: 0.476
[2,    70] loss: 0.374
[2,    80] loss: 0.384
[2,    90] loss: 0.400
[2,   100] loss: 0.402
[2,   110] loss: 0.333
[2,   120] loss: 0.520
[2,   130] loss: 0.344
[2,   140] loss: 0.365
[2,   150] loss: 0.424
[2,   160] loss: 0.687
[2,   170] loss: 0.588
[2,   180] loss: 0.459
[2,   190] loss: 0.420
[2,   200] loss: 0.537
[2,   210] loss: 0.248
[2,   220] loss: 0.370
[2,   230] loss: 0.635
[2,   240] loss: 0.416
[2,   250] loss: 0.384
[2,   260] loss: 0.404
[2,   270] loss: 0.495
[2,   280] loss: 0.333
[2,   290] loss: 0.499
[2,   300] loss: 0.522
[2,   310] loss: 0.613
[2,   320] loss: 0.451
[2,   330] loss: 0.259
[2,   340] loss: 0.222
[2,   350] loss: 0.266
[2,   360] loss: 0.465
[2,   370] loss: 0.370
[2,   380] loss: 0.405
[2,   390] loss: 0.563
[2,   400] loss: 0.459
[2,   410] loss: 0.271
[2,   420] loss: 0.374
[2,   430] loss: 0.438
[2,   440] 

In [16]:
outputs.logits[0][0].item()

0.9415348768234253

In [17]:
evaluate_ce_score(model, eval_dataloader)

<ipython-input-13-e001fe9acb63>:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):


  0%|          | 0/968 [00:00<?, ?it/s]

tensor(0.4239, device='cuda:0')

In [18]:
evaluate_f1_score_huggingface(model, eval_dataloader)

<ipython-input-13-e001fe9acb63>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i, batch in enumerate(tqdm_notebook(eval_dataloader, 0)):


  0%|          | 0/968 [00:00<?, ?it/s]

0.8644111570247934

# Evaluate
create the tsv files

In [19]:
#load dev data
#create predictions on that data
#save the data in tsv
from datasets import Dataset
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
import csv
import os


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=240)#This is the longest that roberta tokenizer would go, dunno

batch_size = 1 #can also be implemented with larger batch_size oc

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased", do_lower_case=True)

filepaths_read = ["/content/ArMIS_dev.json", "/content/ConvAbuse_dev.json", "/content/HS-Brexit_dev.json", "/content/MD-Agreement_dev.json",  "/content/rom_dev.json"]
filepaths_write = ["/content/ArMIS_results.tsv", "/content/ConvAbuse_results.tsv", "/content/HS-Brexit_results.tsv", "/content/MD-Agreement_results.tsv", "/content/romanian_results.tsv"]

epsilon = 1e-12


In [20]:

#print(filepath_read, "a")
for filepath_read, filepath_write in zip(filepaths_read, filepaths_write):
  if os.path.exists(filepath_write):
    os.remove(filepath_write)

  df = pd.read_json(filepath_read)
  df = df.transpose()

  dataset = Dataset.from_pandas(df[["text"]])
  tokenized_dataset = dataset.map(tokenize_function, batched=False)
  tokenized_dataset.set_format("torch")

  dataloader = DataLoader(tokenized_dataset, shuffle=False, batch_size=batch_size)
  with open(filepath_write, 'w', newline='') as tsvfile:
    writer = csv.writer(tsvfile, delimiter='\t', lineterminator='\n')
    for batch in tqdm_notebook(dataloader, 0):
      inputs, attention_mask = batch["input_ids"].to(device), batch["attention_mask"].to(device)
      with torch.no_grad():
        outputs = model(input_ids=inputs, attention_mask = attention_mask)
      logits = outputs.logits

      prediction = torch.argmax(logits, dim=-1)
      probability = torch.softmax(logits, axis=-1)
      probability = torch.clip(probability, epsilon, 1. - epsilon)
      writer.writerow([prediction[0].item(), probability[0][0].item(), probability[0][1].item()])


  0%|          | 0/141 [00:00<?, ?ex/s]

<ipython-input-20-2c596b60d782>:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch in tqdm_notebook(dataloader, 0):


  0%|          | 0/141 [00:00<?, ?it/s]

  0%|          | 0/812 [00:00<?, ?ex/s]

  0%|          | 0/812 [00:00<?, ?it/s]

...

  0%|          | 0/168 [00:00<?, ?it/s]

  0%|          | 0/1104 [00:00<?, ?ex/s]

  0%|          | 0/1104 [00:00<?, ?it/s]

  0%|          | 0/9418 [00:00<?, ?ex/s]

  0%|          | 0/9418 [00:00<?, ?it/s]

In [23]:
from zipfile import ZipFile

filepath = "bert-base-multilingual-cased2ep.zip" 

if os.path.exists(filepath):
    os.remove(filepath)

#loop over filepath names throws an string index out of range for whatever reason(also can't use content here, not sure why)
with ZipFile(filepath, 'w') as zipObj:
  zipObj.write("MD-Agreement_results.tsv")
  zipObj.write("ArMIS_results.tsv")
  zipObj.write("HS-Brexit_results.tsv")
  zipObj.write("ConvAbuse_results.tsv")
  zipObj.write("romanian_results.tsv")
  

In [24]:
from google.colab import files
files.download("bert-base-multilingual-cased2ep.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>